git clone https://github.com/cltl/EMISSOR --branch issue-53-processing
cd EMISSOR
python install.py install

Go to https://github.com/tae898/multimodal-datasets
Run the dataset extraction from there, this will give
raw-video/
    train/ val/ test/
the example_processing expects:
raw-video/
    dia01_....mp4
raw-text/
    dia01_....txt
We need to match the directory structure (copy/rename/link) e.g.
> ln -s original_dir/raw_video/train new_dir/raw_video
Create the docker images in Taes repo:
https://github.com/tae898/face
Checkout the `issue-53-processing` branch from EMISSOR
In the root directory of the EMISSOR repo run
     python -m emissor.processing --plugin example_processing/meld --scenarios example_data/demo --dataset example_data/demo_orig  --num-jobs 4
This runs all steps, you can specify the steps:
    --preprocessing --init --process all
(`all` can be replaced with the processor class name)
* Preprocessing splits videos, copies images and text to "scenarions" folder
* Init creates dia...json, image.json, text.json, without any mentions/annotations
* Process adds mentions/annotations
Parallelize with, max frames per second 
    --num-jobs 4  --fps_max 1
    
    
 python -m emissor.processing --plugin example_processing/meld --scenarios example_data/demo --dataset example_data/demo_orig  --num-jobs 4
    

In [ ]:
#%pip install -r requirements.txt
#For some reason I still had to install
#%pip install importlib-resources

In [1]:
import emissor as em

In [9]:
import os
import spacy
import time
import uuid
from datetime import datetime
from emissor.persistence import ScenarioStorage
from emissor.representation.annotation import AnnotationType, Token, NER
from emissor.representation.container import Index
from emissor.representation.scenario import Modality, ImageSignal, TextSignal, Mention, Annotation, Scenario
import cv2
from datetime import datetime
import tests.test_emissor.test_interaction.driver_util as util

In [11]:
##### Initialisation
agent = "Leolani2"
human = "Stranger"
scenarioid = "myscenario3"
scenario_path = "/Users/piek/Desktop/r-EMISSOR/data"
imagefolder = scenario_path + "/" + scenarioid + "/" + "image"
camera = cv2.VideoCapture(0)
nlp = spacy.load('en_core_web_sm')

scenarioStorage = util.create_scenario(scenario_path, scenarioid)
scenario = scenarioStorage.create_scenario(scenarioid, datetime.now().microsecond, datetime.now().microsecond, agent)

Directory  /Users/piek/Desktop/r-EMISSOR/data/myscenario3  Created 
Directory  /Users/piek/Desktop/r-EMISSOR/data/myscenario3/image  Created 


In [12]:
def add_ner_annotation(signal: TextSignal):
    processor_name = "spaCy"
    utterance = ''.join(signal.seq)

    doc = nlp(utterance)

    offsets, tokens = zip(*[(Index(signal.id, token.idx, token.idx + len(token)), Token.for_string(token.text))
                            for token in doc])

    ents = [NER.for_string(ent.label_) for ent in doc.ents]
    entity_text = [ent.text for ent in doc.ents]
    segments = [token.ruler for token in tokens if token.value in entity_text]

    annotations = [Annotation(AnnotationType.TOKEN.name.lower(), token, processor_name, int(time.time()))
                   for token in tokens]
    ner_annotations = [Annotation(AnnotationType.NER.name.lower(), ent, processor_name, int(time.time()))
                       for ent in ents]

    signal.mentions.extend([Mention(str(uuid.uuid4()), [offset], [annotation])
                            for offset, annotation in zip(offsets, annotations)])
    signal.mentions.extend([Mention(str(uuid.uuid4()), [segment], [annotation])
                            for segment, annotation in zip(segments, ner_annotations)])

    return entity_text

In [13]:

##### First signals to get started
success, frame = camera.read()
imagepath = ""
if success:
    imagepath = imagefolder + "/" + str(datetime.now().microsecond) + ".png"
    cv2.imwrite(imagepath, frame)

#### Initial prompt by the system from which we create a TextSignal and store it
response = "Hi there. Who are you " + human + "?"
print(agent + ": " + response)
textSignal = util.create_text_signal(scenario, response)
scenario.append_signal(textSignal)

utterance = input('\n')
print(human + ": " + utterance)

while not (utterance.lower() == 'stop' or utterance.lower() == 'bye'):
    textSignal = util.create_text_signal(scenario, utterance)
    # @TODO
    ### Apply some processing to the textSignal and add annotations
    entityText = add_ner_annotation(textSignal)

    scenario.append_signal(textSignal)

    if success:
        imageSignal = util.create_image_signal(scenario, imagepath)
        # @TODO
        ### Apply some processing to the imageSignal and add annotations
        ### when done

        scenario.append_signal(imageSignal)

    # Create the response from the system and store this as a new signal

    if not entityText:
        utterance = "Any gossip" + '\n'
    else:
        utterance = "So you what do you want to talk about " + entityText[0] + '\n'

    response = utterance[::-1]
    print(agent + ": " + utterance)
    textSignal = util.create_text_signal(scenario, utterance)
    scenario.append_signal(textSignal)

    ###### Getting the next input signals
    utterance = input('\n')

    success, frame = camera.read()
    if success:
        imagepath = imagefolder + "/" + str(datetime.now().microsecond) + ".png"
        cv2.imwrite(imagepath, frame)

Leolani2: Hi there. Who are you Stranger?



 I am Piek


Stranger: I am Piek
Leolani2: Any gossip




 No


Leolani2: Any gossip




 stop


In [14]:
scenarioStorage.save_scenario(scenario)